In [47]:
# This is a sample Python script.
import pandas as pd
import torch
device = torch.device('cpu')
import dgl
import networkx as nx
import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn as nn
import torch.optim as optim
import dgl.function as fn


In [48]:
df=pd.read_csv("../data/PaySim_kaggle.csv")



Adjust data proportions as the fraud data is heavily biased having less fraudulant cases.

In [49]:
# Set the proportion of non-fraud to remove (e.g., 50% of non-fraud instances)
remove_fraction = 0.9985

# Separate the fraud and non-fraud instances
fraud_df = df[df['isFraud'] == 1]
non_fraud_df = df[df['isFraud'] == 0]

# Randomly sample and remove 'remove_fraction' proportion of non-fraud instances
# Calculate the number of rows to remove based on the fraction
rows_to_remove = int(len(non_fraud_df) * remove_fraction)

# Select the first 'rows_to_remove' rows from the beginning
non_fraud_to_remove = non_fraud_df.tail(rows_to_remove)

# Drop the sampled non-fraud instances from the DataFrame
df = df.drop(non_fraud_to_remove.index)

# Verify the new balance
label_counts = df['isFraud'].value_counts()
fraud_ratio = label_counts[1] / len(df)
non_fraud_ratio = label_counts[0] / len(df)

print(f"Fraud count: {label_counts[1]}")
print(f"Non-fraud count: {label_counts[0]}")
print(f"Fraud ratio: {fraud_ratio:.4f}")
print(f"Non-fraud ratio: {non_fraud_ratio:.4f}")

print(df.head())

Fraud count: 8213
Non-fraud count: 9532
Fraud ratio: 0.4628
Non-fraud ratio: 0.5372
   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [50]:
# Count transactions for each user in nameOrig (senders)
orig_counts = df['nameOrig'].value_counts()

# Count transactions for each user in nameDest (receivers)
dest_counts = df['nameDest'].value_counts()

# Combine the counts for both nameOrig and nameDest
user_counts = orig_counts.add(dest_counts, fill_value=0)

# Filter users with more than one transaction
users_with_multiple_transactions = user_counts[user_counts > 1]

# Print the users who have more than one transaction
if False:
    print(users_with_multiple_transactions)
    print("here")
    print(dest_counts[dest_counts>1])
    print("here")
    print(orig_counts[orig_counts>1])
    print(len(df['nameDest']))
    #print(dest_counts)

In [51]:
# Create a mapping from unique user names to numeric IDs (nodes)
user_mapping = {user: idx for idx, user in enumerate(set(df['nameOrig']).union(set(df['nameDest'])))}

# Create edges between nameOrig and nameDest
src = df['nameOrig'].map(user_mapping).values
dst = df['nameDest'].map(user_mapping).values
if False:
    print("test",len(df.drop_duplicates(subset='nameOrig')))
    print(len(set(df['nameDest'])))
    print(len(set(df['nameOrig']))+len(set(df['nameDest'])))
    print(len(set(df['nameOrig']).union(set(df['nameDest']))))
    print(len(set(set(df['nameOrig']).union(set(df['nameDest'])))))
    print(len(df['nameOrig']))
    idx=0
    for item in df['nameDest']:
        #print(item)
        if 'C985934102' in item:
            idx+=1
    print(idx)
    test=list(df['nameDest'])
    print(len((df['nameDest'])))
    print('test',len(set(test)))

In [52]:
print(src)

[23883 23387  5624 ... 27366  1118  7523]


In [53]:
# Create a DGL graph from the source and destination nodes
g = dgl.graph((src, dst))


In [54]:
# Add transaction amount as edge feature
g.edata['amount'] = torch.tensor(df['amount'].values, dtype=torch.float32)

# Optional: Add fraud information to edge features
g.edata['isFraud'] = torch.tensor(df['isFraud'].values, dtype=torch.float32)

# Initialize node features with zeros (this handles all nodes)
num_nodes = g.num_nodes()
# Group by sender (nameOrig) and calculate the average transaction amount for each user
avg_amount_orig = df.groupby('nameOrig')['amount'].mean()

# Group by receiver (nameDest) and calculate the average transaction amount for each user
avg_amount_dest = df.groupby('nameDest')['amount'].mean()

# Now you can use these averages for your nodes
avg_amount_orig_dict = avg_amount_orig.to_dict()
avg_amount_dest_dict = avg_amount_dest.to_dict()

# Initialize node features (average amounts for both orig and dest users)
avg_amount_orig_nodes = torch.zeros(num_nodes, dtype=torch.float32)
avg_amount_dest_nodes = torch.zeros(num_nodes, dtype=torch.float32)

# Assign average amounts to nodes based on user mapping
for orig_user, avg_amount in avg_amount_orig_dict.items():
    avg_amount_orig_nodes[user_mapping[orig_user]] = avg_amount

for dest_user, avg_amount in avg_amount_dest_dict.items():
    avg_amount_dest_nodes[user_mapping[dest_user]] = avg_amount

# Stack these features and add them to the graph
node_features = torch.stack([avg_amount_orig_nodes, avg_amount_dest_nodes], dim=1)

g.ndata['features'] = node_features

node_labels = torch.zeros(num_nodes, dtype=torch.float32)

# Map 'isFraud' values to corresponding nodes based on user mapping
for user, fraud in df[['nameOrig', 'isFraud']].values:
    node_labels[user_mapping[user]] = fraud

# Map 'isFraud' to destination nodes (nameDest)
for dest_user, fraud in df[['nameDest', 'isFraud']].values:
    node_labels[user_mapping[dest_user]] = fraud
    
node_features = torch.stack([avg_amount_orig_nodes, avg_amount_dest_nodes], dim=1)

g.ndata['features'] = node_features

# Store the fraud labels in g.ndata['isFraud']
g.ndata['isFraud'] = node_labels


71742 43429

In [55]:
g.edata['isFraud'].shape
#node_features.shape
g.num_edges

<bound method DGLGraph.num_edges of Graph(num_nodes=31959, num_edges=17745,
      ndata_schemes={'features': Scheme(shape=(2,), dtype=torch.float32), 'isFraud': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'amount': Scheme(shape=(), dtype=torch.float32), 'isFraud': Scheme(shape=(), dtype=torch.float32)})>

In [56]:
# Print graph information
#print(g)

# Visualize the graph using NetworkX (convert DGL graph to NetworkX for visualization)
#nx_graph = g.to_networkx()

# Optional: Visualize using a layout for better readability
#pos = nx.spring_layout(nx_graph)  # Use a layout for better visualization
#plt.figure(figsize=(12, 12))
#nx.draw(nx_graph, pos, node_size=50, node_color='skyblue', font_size=10, with_labels=True)
#plt.show()

In [57]:
# Assuming that you have edge features like transaction amounts or fraud status
edge_features = g.edata.get('amount', None)  # Assuming 'amount' is an edge feature
if edge_features is not None:
    print("Edge Features (Transaction Amounts):")
    print(edge_features)
else:
    print("No edge features found.")


Edge Features (Transaction Amounts):
tensor([9.8396e+03, 1.8643e+03, 1.8100e+02,  ..., 6.3114e+06, 8.5000e+05,
        8.5000e+05])


In [58]:
# Select 80% of nodes for training
num_train_nodes = int(0.8 * num_nodes)
train_indices = torch.randperm(num_nodes)[:num_train_nodes]  
test_indices = torch.tensor([i for i in range(num_nodes) if i not in train_indices])

# Create train/test masks
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[train_indices] = True
test_mask[test_indices] = True

# Assign to graph
g.ndata['train_mask'] = train_mask
g.ndata['test_mask'] = test_mask


In [59]:
# Define the GraphSAGE model for fraud detection
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GraphSAGE, self).__init__()
        self.layer1 = dgl.nn.SAGEConv(in_feats, hidden_feats, 'mean')
        self.layer2 = dgl.nn.SAGEConv(hidden_feats, out_feats, 'mean')
        self.dropout = nn.Dropout(0.5)
        #self.fc = nn.Linear(out_feats * 2, 1)  # * Concatenate source and destination node embeddings *
        self.fc = nn.Linear(out_feats, 1)  # Output a single value per node (fraud score)
        
    def forward(self, g, features):
        # Apply first GraphSAGE layer and ReLU
        x = self.layer1(g, features)
        x = torch.relu(x)
        x = self.dropout(x)

        # Apply second GraphSAGE layer to get node embeddings
        x = self.layer2(g, x)
        
        # Output a prediction for each node
        logits = self.fc(x).squeeze()  # Output a single value per node
        return logits
        



In [60]:
# Count occurrences of each label in the 'isFraud' column
label_counts = df['isFraud'].value_counts()

# Calculate the proportion of each class
fraud_ratio = label_counts[1] / len(df)  # Assuming '1' represents fraud
non_fraud_ratio = label_counts[0] / len(df)  # Assuming '0' represents non-fraud

# Print the results
print(f"Fraud count: {label_counts[1]}")
print(f"Non-fraud count: {label_counts[0]}")
print(f"Fraud ratio: {fraud_ratio:.4f}")
print(f"Non-fraud ratio: {non_fraud_ratio:.4f}")

Fraud count: 8213
Non-fraud count: 9532
Fraud ratio: 0.4628
Non-fraud ratio: 0.5372


In [61]:
# Instantiate the model
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
in_feats = 2  # balanceOrig and balanceDest features
hidden_feats = 64
out_feats = 1  # Fraud (binary classification)

#pos_weight = torch.tensor([non_fraud_ratio/fraud_ratio],dtype=torch.float)
pos_weight = torch.tensor([1.0],dtype=torch.float)


model = GraphSAGE(in_feats, hidden_feats, out_feats)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model
epochs = 500
for epoch in range(epochs):
    model.train()

    # Forward pass
    logits = model(g, g.ndata['features'])
    #print(logits.shape)

    # Get target labels for the fraud detection task
    labels = g.ndata['isFraud']
    # Compute loss (use train_mask to filter out test edges)
    #print("labels",labels[train_mask].view(-1,1).shape)
    #print("logits", logits[train_mask].view(-1,1).shape)
    loss = loss_fn(logits[train_mask].view(-1,1), labels[train_mask].view(-1,1))
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every few epochs
    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")



Epoch 2/500, Loss: 205846.90625
Epoch 4/500, Loss: 78685.96875
Epoch 6/500, Loss: 31865.6875
Epoch 8/500, Loss: 24021.21484375
Epoch 10/500, Loss: 25621.240234375
Epoch 12/500, Loss: 28713.033203125
Epoch 14/500, Loss: 31108.080078125
Epoch 16/500, Loss: 33092.46484375
Epoch 18/500, Loss: 33154.8203125
Epoch 20/500, Loss: 33238.66015625
Epoch 22/500, Loss: 32864.29296875
Epoch 24/500, Loss: 32042.888671875
Epoch 26/500, Loss: 30097.861328125
Epoch 28/500, Loss: 29340.25390625
Epoch 30/500, Loss: 27079.31640625
Epoch 32/500, Loss: 25707.421875
Epoch 34/500, Loss: 23547.185546875
Epoch 36/500, Loss: 22040.484375
Epoch 38/500, Loss: 21229.841796875
Epoch 40/500, Loss: 19104.486328125
Epoch 42/500, Loss: 17995.50390625
Epoch 44/500, Loss: 16580.5078125
Epoch 46/500, Loss: 15220.8046875
Epoch 48/500, Loss: 14936.611328125
Epoch 50/500, Loss: 13884.0771484375
Epoch 52/500, Loss: 13946.3046875
Epoch 54/500, Loss: 13499.8935546875
Epoch 56/500, Loss: 13130.6484375
Epoch 58/500, Loss: 12817.868

Epoch 2/20, Loss: 50026236.0
Epoch 4/20, Loss: 21910472.0
Epoch 6/20, Loss: 12671362.0
Epoch 8/20, Loss: 12492275.0
Epoch 10/20, Loss: 13373789.0
Epoch 12/20, Loss: 14267830.0
Epoch 14/20, Loss: 14137046.0
Epoch 16/20, Loss: 12949893.0
Epoch 18/20, Loss: 11810090.0
Epoch 20/20, Loss: 10091464.0
0.1
Accuracy: 0.7332
Recall: 0.0086
F1 Score: 0.0073
0.01
Accuracy: 0.7535
Recall: 0.0039
F1 Score: 0.0036
0.2
Accuracy: 0.7378
Recall: 0.0102
F1 Score: 0.0087
0.4
Accuracy: 0.7405
Recall: 0.0079
F1 Score: 0.0068

In [62]:
g.ndata.keys()

dict_keys(['features', 'isFraud', 'train_mask', 'test_mask'])

In [63]:
print(g.ndata['features'])

tensor([[5.1272e+03, 0.0000e+00],
        [0.0000e+00, 5.1361e+05],
        [0.0000e+00, 1.3751e+05],
        ...,
        [2.4206e+03, 0.0000e+00],
        [3.5231e+06, 0.0000e+00],
        [5.3386e+06, 0.0000e+00]])


In [64]:


# Evaluate the model
model.eval()
with torch.no_grad():
    logits = model(g, g.ndata['features'])

    # Convert logits to probabilities
    predictions = torch.sigmoid(logits).squeeze()
    
    # Apply threshold of to classify fraud
    predicted_labels = (predictions > 0.5).float()
    predicted_labels = predicted_labels[test_mask]  # Apply test_mask here
    # Get actual labels
    #true_labels = g.edata['isFraud']
    true_labels = g.ndata['isFraud'][test_mask]  # Apply test_mask here
    #amounts=g.ndata['amount'][test_mask]
    # Store results in a list of dictionaries
    # Convert node indices back to user names
    reverse_user_mapping = {v: k for k, v in user_mapping.items()}  # Reverse mapping
    
    # Retrieve node indices from test_mask
    test_node_indices = test_mask.nonzero().squeeze().tolist()

    node_features_test = g.ndata['features'][test_mask]  # Shape: (num_test_nodes, 2)
    
    avg_amount_orig_test = node_features_test[:, 0].tolist()
    avg_amount_dest_test = node_features_test[:, 1].tolist()
    
    # Map predictions back to transactions with their original balances
    mapped_results = [
        {
            "orig_user": reverse_user_mapping[node],
            "predicted_label": int(pred),
            "true_label": int(true_lab),
            "avg_amount_orig": avg_amount_orig,
            "avg_amount_dest": avg_amount_dest
        }
        for node, pred, true_lab, avg_amount_orig, avg_amount_dest in zip(
            test_node_indices, predicted_labels.tolist(), true_labels.tolist(), avg_amount_orig_test, avg_amount_dest_test
        )
    ]
    
    # Compute accuracy manually
    correct = (predicted_labels == true_labels).sum().item()
    total = true_labels.size(0)
    accuracy = correct / total

    # Print evaluation results
    print(f"Accuracy: {accuracy:.4f}")
    # Calculate F1-Score
    f1 = f1_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    print(f"Recall: {recall:.4f}")
    print(f"Precision:{precision:.4f}")
    print(f"F1 Score: {f1:.4f}")
# Print some mapped results
print("Sample transaction mappings with user names:")
for res in mapped_results[:10]:
    print(res)
    #if res['balance_orig']>0 and res['balance_dest']>0:
    #    print(res)

Accuracy: 0.6189
Recall: 0.6081
Precision:0.6370
F1 Score: 0.6222
Sample transaction mappings with user names:
{'orig_user': 'C1254777952', 'predicted_label': 1, 'true_label': 1, 'avg_amount_orig': 0.0, 'avg_amount_dest': 137508.015625}
{'orig_user': 'C1595140561', 'predicted_label': 1, 'true_label': 1, 'avg_amount_orig': 0.0, 'avg_amount_dest': 57543.33984375}
{'orig_user': 'C699574604', 'predicted_label': 0, 'true_label': 0, 'avg_amount_orig': 6234.93994140625, 'avg_amount_dest': 0.0}
{'orig_user': 'M590399893', 'predicted_label': 1, 'true_label': 0, 'avg_amount_orig': 0.0, 'avg_amount_dest': 8869.7802734375}
{'orig_user': 'C422243042', 'predicted_label': 0, 'true_label': 0, 'avg_amount_orig': 9738.48046875, 'avg_amount_dest': 0.0}
{'orig_user': 'M414634945', 'predicted_label': 1, 'true_label': 0, 'avg_amount_orig': 0.0, 'avg_amount_dest': 760.739990234375}
{'orig_user': 'C1242414689', 'predicted_label': 1, 'true_label': 1, 'avg_amount_orig': 0.0, 'avg_amount_dest': 151105.96875}
{'o

In [41]:
print(node_features)

tensor([[      0.0000,  137508.0156],
        [      0.0000,    9458.1602],
        [  43935.0586,       0.0000],
        ...,
        [1375603.2500,       0.0000],
        [3523089.5000,       0.0000],
        [5338583.5000,       0.0000]])


In [20]:
sum(labels==1)

tensor(16301)